# Atividade 2 - Spark

Esse notebook é baseado no notebook [como instalar pyspark no google colab](https://colab.research.google.com/github/carlosfab/sigmoidal_ai/blob/master/Big_Data_Como_instalar_o_PySpark_no_Google_Colab.ipynb).

O ambiente de execução é configurado para executar no Google Colab.

## Configuração do Spark para Google Colab

In [25]:
# instalar as dependências
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [26]:
# configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

# tornar o pyspark "importável"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

import pyspark

## Exercicios da Atividade 2

### 1. Utilizando o dataset flights.csv, faça operações MapReduce para calcular:

Importação do arquivo necessario para atividade

In [58]:
#sc = pyspark.SparkContext(appName='BigDataAtividade2')

flights_file = '/content/flights.csv'
flights_RDD = sc.textFile(flights_file)
print("dataVoo	id_companhia	num_voos	origem	destino	hora_partida	min_atraso	hora_chegada	min_atraso	duracao	distancia")
print(flights_RDD.take(1))


dataVoo	id_companhia	num_voos	origem	destino	hora_partida	min_atraso	hora_chegada	min_atraso	duracao	distancia
['2014-04-01,19805,1,JFK,LAX,0854,-6.00,1217,2.00,355.00,2475.00']


a. O tempo total de vôo de cada companhia

In [79]:
def tempo_total_de_voo_por_companhia(line):
    record = line.split(',')
    return (record[1],float(record[9]))

voos = flights_RDD.map(tempo_total_de_voo_por_companhia).reduceByKey(lambda a,b: a+b).take(10)

for voo in voos:
  print(f'Companhia: {voo[0]}, tempo total de voo: {voo[1]}')

Companhia: 19930, tempo total de voo: 2001295.0
Companhia: 20409, tempo total de voo: 3009096.0
Companhia: 20366, tempo total de voo: 4142864.0
Companhia: 20437, tempo total de voo: 799733.0
Companhia: 19690, tempo total de voo: 449242.0
Companhia: 20355, tempo total de voo: 4194811.0
Companhia: 21171, tempo total de voo: 916098.0
Companhia: 19805, tempo total de voo: 6273366.0
Companhia: 19790, tempo total de voo: 7747256.0
Companhia: 20436, tempo total de voo: 689589.0


b. Qual é o destino mais visado

In [75]:
def destino_mais_visado(line):
    record = line.split(',')
    return (record[4],int(1))

destino_mais_visado_RDD = flights_RDD.map(destino_mais_visado)
total_destino_mais_visado = destino_mais_visado_RDD.reduceByKey(lambda a,b: a+b).take(500000)

print(f'Quantidade total de destinos: {len(total_destino_mais_visado)}')

destino_mais_visitado = ('', 0)

for destino in total_destino_mais_visado:
  if destino[1] > destino_mais_visitado[1]:
    destino_mais_visitado = destino

print(f'O destino mais visado foi {destino_mais_visitado[0]}, com {destino_mais_visitado[1]} visitas.')

Quantidade total de destinos: 304
O destino mais visado foi ATL, com 30953 visitas.


c. Quais aeroportos cada companhia passou

In [80]:
def aeroportos_de_origem(line):
    record = line.split(',')
    return (record[1],record[3])

def aeroportos_de_destino(line):
    record = line.split(',')
    return (record[1],record[4])

origens = flights_RDD.map(aeroportos_de_origem).take(50000)
destinos = flights_RDD.map(aeroportos_de_destino).take(50000)

origens_RDD = sc.parallelize(origens)
destinos_RDD = sc.parallelize(destinos)

grouped_elementsDestino = origens_RDD.union(destinos_RDD).distinct().groupByKey().take(50000)

for key, values in grouped_elementsDestino:
    print(key, list(values))

19930 ['DCA', 'EWR', 'BOS', 'PDX', 'FLL', 'ADQ', 'FAI', 'PSP', 'SNA', 'ONT', 'PHX', 'AUS', 'ATL', 'PHL', 'ANC', 'SCC', 'BRW', 'KTN', 'CDV', 'SIT', 'WRG', 'PSG', 'SAN', 'OAK', 'SMF', 'BUR', 'BLI', 'DFW', 'SLC', 'STL', 'SEA', 'LAX', 'ORD', 'MSP', 'JNU', 'YAK', 'OME', 'OTZ', 'SFO', 'LAS', 'DEN', 'IAH', 'KOA', 'ADK', 'MCO', 'BET', 'SJC', 'TUS', 'SAT', 'GEG', 'MCI', 'OGG', 'LIH', 'HNL']
20409 ['DCA', 'FLL', 'BOS', 'SJU', 'SWF', 'EWR', 'JAX', 'CLT', 'TPA', 'HOU', 'BTV', 'CHS', 'DTW', 'RIC', 'SAV', 'AUS', 'BQN', 'PVD', 'PHX', 'PDX', 'SRQ', 'PHL', 'BUF', 'STX', 'HPN', 'DFW', 'BWI', 'SAN', 'PWM', 'LGA', 'OAK', 'SMF', 'SLC', 'ABQ', 'BUR', 'STT', 'RSW', 'PBI', 'IAD', 'ORH', 'PIT', 'ORD', 'ROC', 'LAS', 'LGB', 'DEN', 'SEA', 'SFO', 'LAX', 'JFK', 'MCO', 'MSY', 'RDU', 'SYR', 'BDL', 'PSE', 'SJC']
21171 ['EWR', 'AUS', 'DCA', 'PSP', 'FLL', 'BOS', 'PDX', 'PHL', 'SAN', 'DFW', 'LAX', 'SFO', 'SEA', 'ORD', 'LAS', 'IAD', 'JFK', 'SJC', 'MCO']
19805 ['DFW', 'MIA', 'LGA', 'SLC', 'STL', 'BWI', 'SAN', 'ABQ', 'RNO',

d. O vôo de maior distância de cada companhia

In [35]:
def distancias_por_companhia(line):
    record = line.split(',')
    return (record[1],float(record[10]))

distancias = flights_RDD.map(distancias_por_companhia).collect()
maxDist_RDD = sc.parallelize(distancias).reduceByKey(max).collect()
print(maxDist_RDD)

[('19930', 2874.0), ('20409', 2704.0), ('20366', 1345.0), ('20437', 2139.0), ('19690', 4983.0), ('20355', 2979.0), ('21171', 2704.0), ('19805', 3784.0), ('19790', 4502.0), ('20436', 1703.0), ('20398', 1379.0), ('20304', 1535.0), ('19977', 4962.0), ('19393', 2335.0)]


e. Qual é o vôo mais frequente de cada companhia

In [83]:
voos_frequentes = {}
quantidade_voos_frequentes = {}

def mais_frequente(line):
    record = line.split(',')
    return (record[1], record[2])

voos_mais_frequentes_por_companhia = flights_RDD.map(mais_frequente).take(500000)
voos_mais_frequentes_por_companhia_RDD = sc.parallelize(voos_mais_frequentes_por_companhia)

voos_mais_frequentes_por_companhia_agrupado_RDD = voos_mais_frequentes_por_companhia_RDD.groupByKey().sortByKey()

for i in voos_mais_frequentes_por_companhia_agrupado_RDD.collect():
    voos_frequentes[i[0]] = list(i[1])
    
for companhia in voos_frequentes.keys():
    for id_flight in voos_frequentes[companhia]:
        if id_flight in quantidade_voos_frequentes:
            quantidade_voos_frequentes[id_flight] += 1
        else:
            quantidade_voos_frequentes[id_flight] = 1
            
    quantidade_de_voos = int(max(quantidade_voos_frequentes.values()))
    id_do_voo = list(quantidade_voos_frequentes.keys())[list(quantidade_voos_frequentes.values()).index(quantidade_de_voos)]
    quantidade_voos_frequentes.clear()
    
    print(f'O voo mais frequente da companhia: {companhia} é {id_do_voo} com {quantidade_de_voos} voos')

O voo mais frequente da companhia: 19393 é 583 com 170 voos
O voo mais frequente da companhia: 19690 é 1 com 30 voos
O voo mais frequente da companhia: 19790 é 1258 com 60 voos
O voo mais frequente da companhia: 19805 é 29 com 60 voos
O voo mais frequente da companhia: 19930 é 64 com 150 voos
O voo mais frequente da companhia: 19977 é 1246 com 75 voos
O voo mais frequente da companhia: 20304 é 5325 com 90 voos
O voo mais frequente da companhia: 20355 é 409 com 120 voos
O voo mais frequente da companhia: 20366 é 2514 com 60 voos
O voo mais frequente da companhia: 20398 é 2726 com 60 voos
O voo mais frequente da companhia: 20409 é 2054 com 59 voos
O voo mais frequente da companhia: 20436 é 143 com 50 voos
O voo mais frequente da companhia: 20437 é 221 com 46 voos
O voo mais frequente da companhia: 21171 é 251 com 60 voos


### 2. Utilizando o dataset shakespeare.txt, faça uma operação de MapReduce para construir um índice invertido contendo como chave cada palavra e como valor uma lista de linhas onde ela foi mecionada.

In [82]:
#text_file = sc.textFile("hdfs://...")
#counts = text_file.flatMap(lambda line: line.split(" ")) \
#             .map(lambda word: (word, 1)) \
#             .reduceByKey(lambda a, b: a + b)
#counts.saveAsTextFile("hdfs://...")